In [3]:
#These are the libraries you can use.  You may add any libraries directy related to threading if this is a direction
#you wish to go (this is not from the course, so it's entirely on you if you wish to use threading).  Any
#further libraries you wish to use you must email me, james@uwaterloo.ca, for permission.

from IPython.display import display, Math, Latex

import pandas as pd
import numpy as np
import numpy_financial as npf
import yfinance as yf
import matplotlib.pyplot as plt
import random
from datetime import datetime

In [4]:
print('hello there')

hello there


## Group Assignment
### Team Number: 12
### Team Member Names: Sharuga, Derek, Alex
### Team Strategy Chosen: Market Beat

Disclose any use of AI for this assignment below (detail where and how you used it).  Please see the course outline for acceptable uses of AI.


In [13]:
# # Function to calculate the Sharpe ratio for a given stock
# def calculate_sharpe_ratio(stock_data, risk_free_rate=0.01):
#     stock_data['returns'] = stock_data['Close'].pct_change()
#     avg_return = stock_data['returns'].mean()
#     std_dev = stock_data['returns'].std()
    
#     # Avoid division by zero
#     if std_dev == 0:
#         return 0
#     sharpe_ratio = (avg_return - risk_free_rate) / std_dev
#     return sharpe_ratio

# # Function to compare correlations and add stocks to the portfolio
# def compare_corr(portfolio_data, new_stock_data, threshold=0.5):
#     new_stock_returns = new_stock_data['returns']
    
#     for stock_data in portfolio_data:
#         portfolio_returns = stock_data['returns']
#         correlation = portfolio_returns.corr(new_stock_returns)
        
#         # If correlation is below the threshold, add the stock to the portfolio
#         if correlation > threshold:
#             return False
#     return True

# # Main stock screener function
# def stock_screener(ticker_list):
#     portfolio = []
    
#     for ticker in ticker_list:
#         stock_data = yf.Ticker(ticker).history(start='2021-01-04', end='2024-03-01', interval='1d')
#         sharpe_ratio = calculate_sharpe_ratio(stock_data)
#         if not portfolio:
#             portfolio.append({'ticker': ticker, 'sharpe_ratio': sharpe_ratio, 'data': stock_data})
        
#         else:
#             highest_sharpe_stock = max(portfolio, key=lambda x: x['sharpe_ratio'])
            
#             if sharpe_ratio > highest_sharpe_stock['sharpe_ratio'] and compare_corr([stock['data'] for stock in portfolio], stock_data):
#                 portfolio.append({'ticker': ticker, 'sharpe_ratio': sharpe_ratio, 'data': stock_data})

#     return portfolio

# # Assuming the tickers are in a CSV file provided
# all_data = pd.read_csv('Tickers_Example.csv')  # Ensure it has columns like 'Ticker' and 'Close'
# ticker_list = all_data.iloc[:, -1].tolist()  # Rightmost column for tickers
# portfolio = stock_screener(ticker_list)

# for stock in portfolio:
#     print(f"Ticker: {stock['ticker']} - Sharpe Ratio: {stock['sharpe_ratio']}")
all_data.head()

,AAPL
0,ABBV
1,ABT
2,ACN
3,AGN
4,AIG


In [39]:

def validity(tickers):
    final_list = []
    for ticker in tickers:
        if (not check_delist(ticker) and
            check_currency(ticker) and
            check_trading_days(ticker)):
                final_list.append(ticker)
    return final_list

def check_delist(ticker):
    stock = yf.Ticker(ticker)
    try:
        data = stock.history(period='1d')  # Fetch recent data
        if data.empty:
            # print(f"  {ticker} is delisted (no data).")
            return True  # No data means it could be delisted
        else:
            # Ensure that there is actually valid market data for this stock - AI
            if 'Close' not in data.columns or data['Close'].isnull().all():
                # print(f"  {ticker} has no valid close prices.")
                return False  # No valid close prices could also indicate delisting
            return False  # Valid stock with data
    except Exception as e:
        print(f"  Error checking {ticker} delist status: {e}")
        return True  # Error in fetching data; assume delisted or unavailable

def check_trading_days(ticker):
    min_days=18
    """Check if a stock has at least 18 trading days in the last month."""
    stock = yf.Ticker(ticker)
    try:
        data = stock.history(period="1mo")  # Last month
        #AI
        trading_days = data.shape[0]
        return trading_days >= min_days  # Must have at least `min_days` trading days
    except Exception as e:
        return False  # If error, assume it’s invalid

def check_currency(ticker):
    """Check if the stock is in USD or CAD currency."""
    stock = yf.Ticker(ticker)
    try:
        currency = stock.info.get('currency', None)
        return currency in ['USD', 'CAD']
    except Exception as e:
        return False  # If error, assume it’s invalid


# Load the CSV file without headers -- AI
all_data = pd.read_csv('Tickers_Example.csv', header=None)

# Passes the first (and only) column to the function
validity(all_data[0])


$AGN: possibly delisted; no price data found  (period=1d) (Yahoo error = "No data found, symbol may be delisted")
$CELG: possibly delisted; no price data found  (period=1d) (Yahoo error = "No data found, symbol may be delisted")
$MON: possibly delisted; no price data found  (period=1d) (Yahoo error = "No data found, symbol may be delisted")
$RTN: possibly delisted; no price data found  (period=1d) (Yahoo error = "No data found, symbol may be delisted")


['AAPL',
 'ABBV',
 'ABT',
 'ACN',
 'AIG',
 'AMZN',
 'AXP',
 'BA',
 'BAC',
 'BB.TO',
 'BIIB',
 'BK',
 'BLK',
 'BMY',
 'C',
 'CAT',
 'CL',
 'KO',
 'LLY',
 'LMT',
 'MO',
 'MRK',
 'PEP',
 'PFE',
 'PG',
 'PM',
 'PYPL',
 'QCOM',
 'RY.TO',
 'SHOP.TO',
 'T.TO',
 'TD.TO',
 'TXN',
 'UNH',
 'UNP',
 'UPS',
 'USB']

## Contribution Declaration

The following team members made a meaningful contribution to this assignment:

Insert Names Here.